In [1]:
from __future__ import print_function, division

import numpy as np
import os
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import sklearn.model_selection as model_selection

In [2]:
# Import data set
dataset = []
with open('nparr_spec_image.pickle','rb') as modelFile:
     dataset = pickle.load(modelFile)

In [3]:
print(dataset[0][0].shape)

(303, 722)


In [4]:
print(np.matrix(dataset[0][0]))

[[166 176 176 ... 241 241 241]
 [165 175 175 ... 255 255 255]
 [165 175 175 ... 255 255 255]
 ...
 [238 252 252 ... 255 255 255]
 [237 251 251 ... 255 255 255]
 [237 251 251 ... 255 255 255]]


# Import Model

In [ ]:
# Initialize ResNet model with initial weights
tf_model = models.resnet18(weights = models.ResNet18_Weights.DEFAULT)

In [101]:
print(tf_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Modify initial and last layers to allow model to adapt to our inputs/outputs.

In [102]:
# Get last layer and reset last linear layer
last_name, last_layer = list(tf_model.named_modules())[-1]
tf_model.fc = nn.Linear(in_features=512, out_features=10, bias=True)

# Prepend initial layers
tf_model.conv1 = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size= (9,9), stride=(4,4), padding=(3,3)),
    nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU(inplace=True),
    tf_model.conv1
)
print(tf_model.named_modules)

<bound method Module.named_modules of ResNet(
  (conv1): Sequential(
    (0): Conv2d(1, 3, kernel_size=(9, 9), stride=(4, 4), padding=(3, 3))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

# Create Dataloaders

In [134]:
classes = ['bird', 'cat', 'chicken', 'cow', 'dog', 'donkey', 'frog', 'lion', 'monkey', 'sheep']
import operator
batch_size = 4

# Used from Google Gemini
from torch.utils.data import Dataset
class DAN_Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = torch.tensor(data)
        self.labels = torch.tensor(labels)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


def fetch_dataloader(data, batch_size, train_transform=None, val_transform=None):
    '''
    Inputs:
    :param data: Data to load into DataLoaders.
    :param batch_size: Batch size for DataLoaders.
    :param train_transform: Transforms to apply to train Dataset
    :param val_transform: Transforms to apply to validation Dataset
    '''
    # https://www.geeksforgeeks.org/python-get-first-element-in-list-of-tuples/
    images = list(map(operator.itemgetter(0), dataset))
    images = np.array(images).astype(np.float32)
    # Grab lowercase labels and convert to integers
    labels = list(map(operator.itemgetter(1), dataset))
    labels = list(map(lambda x: x.lower(), labels))
    labels = list(map(lambda x: classes.index(x), labels))

    # Split into train and validation sets.
    x_train, x_val, y_train, y_val  = model_selection.train_test_split(images, labels, train_size=0.8, shuffle=True)
    # Convert to Dataset
    train_data = DAN_Dataset(x_train, y_train, train_transform)
    val_data = DAN_Dataset(x_val, y_val, val_transform)

    # Separate every 10th value
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=2)
    return train_loader, val_loader


train_transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.ConvertImageDtype(torch.float32),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.ConvertImageDtype(torch.float32),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [135]:
train, val = fetch_dataloader(dataset, batch_size, train_transform, val_transform)

# Training

In [136]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tf_model.to(device)
tf_model.train()

# Set the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(tf_model.parameters(), lr=0.001, momentum=0.9)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [137]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.reshape(inputs, (4, 1, 303, 722))
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = tf_model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

Finished Training


# Evaluate Model

In [138]:
# Gemini generated code with google search: "pytorch evaluate accuracy of model"
from torchmetrics import Accuracy

def evaluate_model(model, data_val, classes, device):
    model.eval()
    predictions = []
    accuracy = Accuracy(task='multiclass', num_classes=classes).to(device)
    with torch.no_grad():
        for i, data in enumerate(data_val, 0):
            # get the inputs; data is a list of [inputs, labels]
            try:
                inputs, labels = data
                inputs = torch.reshape(inputs, (inputs.shape[0], 1, 303, 722))
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                predictions.append(outputs)
                accuracy.update(outputs, labels)
            except:
                continue
        return accuracy.compute(), predictions

In [140]:
predictions = []
accuracy, predictions = evaluate_model(tf_model, val, 10, device)
print("Accuracy: ", accuracy)

pred_labels = []
for batch in predictions:
    for prediction in batch:
        x = (prediction == torch.max(prediction)).nonzero(as_tuple=False)[0]
        pred_labels.append(x)
pred_labels = list(map(lambda x: x.cpu().numpy().item(), pred_labels))
print(pred_labels)
print(classes)

Accuracy:  tensor(0.9771, device='cuda:0')
[4, 1, 0, 7, 0, 1, 4, 1, 1, 0, 4, 1, 0, 1, 9, 0, 1, 1, 4, 3, 1, 3, 9, 4, 1, 6, 0, 4, 2, 0, 6, 1, 1, 2, 3, 0, 2, 4, 0, 6, 8, 4, 6, 0, 7, 9, 3, 9, 4, 4, 1, 4, 7, 7, 9, 0, 4, 1, 4, 4, 1, 3, 1, 1, 0, 6, 3, 1, 6, 1, 1, 6, 1, 9, 1, 1, 0, 4, 0, 4, 0, 0, 6, 4, 4, 1, 0, 0, 1, 9, 4, 4, 4, 1, 9, 4, 7, 4, 5, 0, 1, 1, 0, 4, 8, 0, 0, 3, 3, 0, 1, 1, 4, 7, 0, 4, 1, 0, 4, 3, 3, 1, 0, 2, 7, 4, 8, 1, 3, 1, 1, 0, 0, 3, 1, 0, 1, 0, 0, 2, 0, 0, 2, 8, 6, 1, 9, 7, 2, 0, 0, 1, 0, 8, 1, 1, 9, 0, 3, 3, 0, 4, 4, 4, 0, 4, 3, 1, 7, 3, 0, 4, 4, 4, 0]
['bird', 'cat', 'chicken', 'cow', 'dog', 'donkey', 'frog', 'lion', 'monkey', 'sheep']


In [141]:
from torchsummary import summary

summary(tf_model, (1, 303, 722))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 3, 76, 180]             246
       BatchNorm2d-2           [-1, 3, 76, 180]               6
              ReLU-3           [-1, 3, 76, 180]               0
            Conv2d-4           [-1, 64, 38, 90]           9,408
       BatchNorm2d-5           [-1, 64, 38, 90]             128
              ReLU-6           [-1, 64, 38, 90]               0
         MaxPool2d-7           [-1, 64, 19, 45]               0
            Conv2d-8           [-1, 64, 19, 45]          36,864
       BatchNorm2d-9           [-1, 64, 19, 45]             128
             ReLU-10           [-1, 64, 19, 45]               0
           Conv2d-11           [-1, 64, 19, 45]          36,864
      BatchNorm2d-12           [-1, 64, 19, 45]             128
             ReLU-13           [-1, 64, 19, 45]               0
       BasicBlock-14           [-1, 64,